In [1]:
import numpy as np
import pandas as pd
from Bio.Seq import Seq

In [2]:
df = pd.read_table('genes.txt', sep='\t', header=None, names=['Species', 'Gene', 'Sequence'])

In [3]:
df.head()

,Species,Gene,Sequence
0,Silurus_microdorsalis,ND1,ATGTTAGCTCTACTAATAACACATGTAATTAACCCCTTAGCCTATA...
1,Silurus_microdorsalis,ND2,ATGAGCCCCTACGTCATTACAATTCTCCTATCAAGCCTCGGCCTAG...
2,Silurus_microdorsalis,COX1,GTGACAATCACGCGCTGATTCTTCTCAACCAACCATAAAGACATTG...
3,Silurus_microdorsalis,COX2,ATGGCACACCCCTCACAACTAGGATTCCAAGACGCGGCCTCCCCTG...
4,Silurus_microdorsalis,ATP8,ATGCCACAATTAAACCCCGCCCCATGATTTGCAATTCTTGTATTCT...


In [4]:
def get_fourfold_deg_sites(seq, vec_of_syn):
    codon_list = []
    third_pos = ''
    for i in range(3, len(seq), 3):
        codon = seq[i-3:i]
        if codon in vec_of_syn:
            codon_list.append(codon)
            third_pos += codon[-1]
    return(third_pos, len(codon_list))

In [21]:
VecOfSynFourFoldDegenerateSites = ['CTT','CTC','CTA','CTG','GTT','GTC','GTA','GTG','TCT','TCC','TCA','TCG','CCT','CCC','CCA','CCG','ACT','ACC','ACA','ACG','GCT','GCC','GCA','GCG','CGT','CGC','CGA','CGG','GGT','GGC','GGA','GGG']
df['ThirdPos'], df['SitesNumber'] = df.apply(lambda r: get_fourfold_deg_sites(r['Sequence'], VecOfSynFourFoldDegenerateSites), axis=1)

ValueError: too many values to unpack (expected 2)

In [6]:
VecOfSynFourFoldDegenerateSites = ['CTT','CTC','CTA','CTG','GTT','GTC','GTA','GTG','TCT','TCC','TCA','TCG','CCT','CCC','CCA','CCG','ACT','ACC','ACA','ACG','GCT','GCC','GCA','GCG','CGT','CGC','CGA','CGG','GGT','GGC','GGA','GGG']

a = []
b = []
for seq in df.Sequence:
    first, second = get_fourfold_deg_sites(seq, VecOfSynFourFoldDegenerateSites)
    a.append(first)
    b.append(second)

In [7]:
df['ThirdPos'], df['SitesNumber'] = a, b

In [8]:
df.head()

,Species,Gene,Sequence,ThirdPos,SitesNumber
0,Silurus_microdorsalis,ND1,ATGTTAGCTCTACTAATAACACATGTAATTAACCCCTTAGCCTATA...,TAAAACCACAAATCCAAAAATCAAACTGAACAATACACCCCCCCCT...,190
1,Silurus_microdorsalis,ND2,ATGAGCCCCTACGTCATTACAATTCTCCTATCAAGCCTCGGCCTAG...,CCACAACCACACCACCCAAACCCCAAACCTACTCAAAACCTAAAAC...,198
2,Silurus_microdorsalis,COX1,GTGACAATCACGCGCTGATTCTTCTCAACCAACCATAAAGACATTG...,GAGCACCCTATCCACCACACAAGCTCCCACTTGTCAAAGAGTATTG...,283
3,Silurus_microdorsalis,COX2,ATGGCACACCCCTCACAACTAGGATTCCAAGACGCGGCCTCCCCTG...,ACAAAGCCTATCCTAATATACCTCCACCGAAAATCCCCAAATCTTA...,109
4,Silurus_microdorsalis,ATP8,ATGCCACAATTAAACCCCGCCCCATGATTTGCAATTCTTGTATTCT...,ACCAATAGAAAACTCATACCTTACA,25


In [31]:
df[df['Gene'] == 'ATP8'].describe()

,SitesNumber
count,3951.000000
mean,26.051886
std,4.810073
min,5.000000
25%,23.000000
50%,26.000000
75%,29.000000
max,132.000000


In [32]:
df[df['Gene'] == 'CytB'].describe()

,SitesNumber
count,3954.000000
mean,200.381892
std,11.975218
min,0.000000
25%,194.000000
50%,202.000000
75%,209.000000
max,225.000000


In [9]:
def gc_skew(seq):
    g = seq.count('G')
    c = seq.count('C')
    skew = (g - c) / (g + c)
    return(skew)

def at_skew(seq):
    a = seq.count('A')
    t = seq.count('T')
    skew = (a - t) / (a + t)
    return(skew)

In [27]:
gc_list = []
at_list = []
for i in df.ThirdPos:
    try:
        gc_list.append(gc_skew(i))
    except ZeroDivisionError:
        gc_list.append(0)
    try:
        at_list.append(at_skew(i))
    except ZeroDivisionError:
        at_list.append(0)

In [29]:
df['GCSkew'], df['ATSkew'] = gc_list, at_list

In [30]:
df[df['Gene'] == 'CytB'].describe()

,SitesNumber,GCSkew,ATSkew
count,3954.000000,3954.000000,3954.000000
mean,200.381892,-0.698790,0.449214
std,11.975218,0.177346,0.216323
min,0.000000,-1.000000,-0.551020
25%,194.000000,-0.817718,0.306571
50%,202.000000,-0.731707,0.479333
75%,209.000000,-0.617978,0.619835
max,225.000000,0.777778,0.886525


In [31]:
df[df['Gene'] == 'ATP8'].describe()

,SitesNumber,GCSkew,ATSkew
count,3951.000000,3951.000000,3951.000000
mean,26.051886,-0.678314,0.283611
std,4.810073,0.310572,0.278132
min,5.000000,-1.000000,-1.000000
25%,23.000000,-1.000000,0.090909
50%,26.000000,-0.750000,0.294118
75%,29.000000,-0.500000,0.478261
max,132.000000,1.000000,1.000000


In [40]:
df.loc[:, ['Species', 'Gene', 'Sites_Number', 'GCSkew', 'ATSkew']].to_csv('cds_at_gc_skew_with_sites_number_python.csv', 
                                                                          sep='\t')

In [41]:
df.head()

,Species,Gene,Sequence,ThirdPos,SitesNumber,GCSkew,ATSkew
0,Silurus_microdorsalis,ND1,ATGTTAGCTCTACTAATAACACATGTAATTAACCCCTTAGCCTATA...,TAAAACCACAAATCCAAAAATCAAACTGAACAATACACCCCCCCCT...,190,-0.789474,0.561404
1,Silurus_microdorsalis,ND2,ATGAGCCCCTACGTCATTACAATTCTCCTATCAAGCCTCGGCCTAG...,CCACAACCACACCACCCAAACCCCAAACCTACTCAAAACCTAAAAC...,198,-0.846154,0.682243
2,Silurus_microdorsalis,COX1,GTGACAATCACGCGCTGATTCTTCTCAACCAACCATAAAGACATTG...,GAGCACCCTATCCACCACACAAGCTCCCACTTGTCAAAGAGTATTG...,283,-0.415094,0.389831
3,Silurus_microdorsalis,COX2,ATGGCACACCCCTCACAACTAGGATTCCAAGACGCGGCCTCCCCTG...,ACAAAGCCTATCCTAATATACCTCCACCGAAAATCCCCAAATCTTA...,109,-0.695652,0.428571
4,Silurus_microdorsalis,ATP8,ATGCCACAATTAAACCCCGCCCCATGATTTGCAATTCTTGTATTCT...,ACCAATAGAAAACTCATACCTTACA,25,-0.750000,0.411765
